In [1]:
import pandas as pd 


In [2]:
# Indonesia Data including '[Units]' Data 
indonesia_full['[Units]'].value_counts()

PERCENT                   638
NUMBER                    354
PER_100000_POP             69
INDEX                      65
CU_USD                     50
KMSQ                       47
CON_USD                    33
NUM_TH                     30
Ratio                      29
CU_USD_M                   24
PER_1000_LIVE_BIRTHS       24
CON_USD_M                  20
HA_TH                      18
W_PER_CAPITA               16
SCORE                       8
PER_100_POP                 8
TEU                         8
NUM_M                       8
PER_1000_POP                8
CUR_LCU_M                   8
MJ_PER_GDP_CON_PPP_USD      7
TONNES_M                    7
USD/m3                      7
KG_PER_CON_USD              7
PER_POP_U5                  6
BOOL                        6
TONNES                      6
T_PER_HA                    6
mgr/m^3                     5
PER_1000000_POP             3
Name: [Units], dtype: int64

In [76]:

indonesia_full[indonesia_full['[Units]']=='SCORE']

,Unnamed: 0,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,...,[Activity],[Nature],[Sampling Stations],[Cities],[Level of requirement],[Quantile],[IHR Capacity],[Hazard type],[Migratory status],[Name of international institution]
481,1566,5.0,5.a,5.a.2,SG_LGL_LNDFEMOD,Degree to which the legal framework (including...,360.0,Indonesia,2019.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,1567,5.0,5.a,5.a.2,SG_LGL_LNDFEMOD,Degree to which the legal framework (including...,360.0,Indonesia,2020.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,2443,8.0,8.8,8.8.2,SL_LBR_NTLCPL,Level of national compliance with labour right...,360.0,Indonesia,2015.0,4.15,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,2444,8.0,8.8,8.8.2,SL_LBR_NTLCPL,Level of national compliance with labour right...,360.0,Indonesia,2016.0,2.67,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
727,2445,8.0,8.8,8.8.2,SL_LBR_NTLCPL,Level of national compliance with labour right...,360.0,Indonesia,2017.0,2.68,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
728,2446,8.0,8.8,8.8.2,SL_LBR_NTLCPL,Level of national compliance with labour right...,360.0,Indonesia,2018.0,2.68,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,2469,8.0,8.b,8.b.1,SL_CPA_YEMP,Existence of a developed and operationalized n...,360.0,Indonesia,2019.0,2.00,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
736,2470,8.0,8.b,8.b.1,SL_CPA_YEMP,Existence of a developed and operationalized n...,360.0,Indonesia,2020.0,2.00,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
import os
import glob

path_g = '../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Guatemala/'

extension = 'csv'

# list of paths
guatemala_paths = glob.glob(path_g+'*.{}'.format(extension))

# list of csv's (no path)
guatemala_list = []
for string in guatemala_paths: 
    string = string.replace(path_g,'')
    guatemala_list.append(string)

# Indonesia 
path_i = '../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Indonesia/'

# list of paths
indonesia_paths = glob.glob(path_i+'*.{}'.format(extension))

# list of csv's (no path)
indonesia_list = []
for string in indonesia_paths: 
    string = string.replace(path_i,'')
    indonesia_list.append(string)

In [86]:
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np

indonesia_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Indonesia%20Data%20Without%20Disaggregation.csv')
series_units = indonesia_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()

path_i_out = '../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Indonesia/'
for csv in indonesia_list: 
    df = pd.read_csv(path_i+csv)
    for col in df.loc[:, df.columns != 'Year']:
        if (df[col].notnull().sum(axis=0) > 1) & (df[col].isnull().sum(axis=0) > 1):
            missing_rows = df[df[col].isnull()]
            nonmissing_rows = df[df[col].notnull()]
            missing_years = missing_rows[['Year']]
            y = nonmissing_rows[col]
            X = nonmissing_rows[['Year']]
            lm = LinearRegression().fit(X,y)
            missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
            missing_rows["inferred_col"] = lm.predict(missing_years)
            missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
            for i in np.logspace(0,10,10):
                if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PER_'+str(i)+'_POP') is True: 
                    missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=i)
            if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PERCENT') is True:
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=100)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='SCORE') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=10)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='Ratio') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=1)
            missing_rows = missing_rows[["Year","inferred_col"]]
            df = df.merge(missing_rows, on='Year', how ='left')
            df[str(col)+'_new'] = np.where(df[col].isnull(),df["inferred_col"],df[col])
            df = df.drop(columns = ["inferred_col", str(col)])
        elif df[col].notnull().sum(axis=0) <= 1:
            df = df.drop(columns = [str(col)])
    df.to_csv(path_i_out + csv)

<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

...

<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-86-0b6ed7c8498c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-86-0b6ed7c8498c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-86-0b6ed7c8498c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [87]:
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np

guatemala_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Guatmala%20Data%20Without%20Disaggregation.csv')
series_units = guatemala_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()

path_g_out = '../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Guatemala/'
for csv in guatemala_list: 
    df = pd.read_csv(path_g+csv)
    for col in df.loc[:, df.columns != 'Year']:
        if (df[col].notnull().sum(axis=0) > 1) & (df[col].isnull().sum(axis=0) > 1):
            missing_rows = df[df[col].isnull()]
            nonmissing_rows = df[df[col].notnull()]
            missing_years = missing_rows[['Year']]
            y = nonmissing_rows[col]
            X = nonmissing_rows[['Year']]
            lm = LinearRegression().fit(X,y)
            missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
            missing_rows["inferred_col"] = lm.predict(missing_years)
            missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
            for i in np.logspace(0,10,10):
                if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PER_'+str(i)+'_POP') is True: 
                    missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=i)
            if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PERCENT') is True:
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=100)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='SCORE') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=10)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='Ratio') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=1)
            missing_rows = missing_rows[["Year","inferred_col"]]
            df = df.merge(missing_rows, on='Year', how ='left')
            df[str(col)+'_new'] = np.where(df[col].isnull(),df["inferred_col"],df[col])
            df = df.drop(columns = ["inferred_col", str(col)])
        elif df[col].notnull().sum(axis=0) <= 1:
            df = df.drop(columns = [str(col)])
    df.to_csv(path_g_out + csv)
    

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

...

<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-87-921f5dad5f0d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-87-921f5dad5f0d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-87-921f5dad5f0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati